In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ML4FINANCE

/content/drive/MyDrive/ML4FINANCE


In [ ]:
import torch
from transformers import pipeline
from sentiment_analysis import *

# 10-K Reports

In [ ]:
tenkreports = pd.read_parquet('datasets/Predictors/mda_text.parquet')

In [ ]:
tenkreports

In [ ]:
tenkreports['report_year'].value_counts()

We only have data for 2015.

# Chen-Zimmerman data

In [ ]:
chenz = pd.read_csv('datasets/Predictors/chenz.csv')
chenz


# Earning Calls

In [ ]:
earnings_calls = pd.read_parquet('datasets/Predictors/earnings_calls.parquet')

In [ ]:
earnings_calls

In [ ]:
earnings_calls.dtypes

# Quarterly Compustat Firm Characteristics

In [ ]:
firm_char = pd.read_csv('datasets/Predictors/CompFirmCharac.csv')

In [ ]:
firm_char

# JKP

In [ ]:
jkp = pd.read_csv('datasets/Predictors/jkp.csv')

In [ ]:
jkp

idée: C'est que le sentiment analysis va retourner uniquement un sentiment: positif, négatif ou neutre donc il n'y aura pas vraiment de data leakage. Dans les faits, dans certains datasets ils parlent des performances mais on est pas dans le data leakage car notre sentiment analysis ne resort pas de chiffre non?

# Earning Calls

In [ ]:
earnings_calls = pd.read_parquet('datasets/Predictors/earnings_calls.parquet')

In [ ]:
earnings_calls

,transcriptid,componentorder,componenttext,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename
0,3285682,0,"Good morning, everyone, and welcome to the LXI...",2020-11-23,323562,LXI REIT plc,Presentation Operator Message
1,50630,5,"Thanks, Bob. In summary 2009 was a pivotal yea...",2010-02-18,063083,Endologix LLC,Presenter Speech
2,840499,103,"Just one thing left to ask, actually. If we ta...",2015-07-28,002410,BP p.l.c.,Question
3,47300,58,"I said, I think it certainly has been a rare o...",2010-01-27,009317,SEI Investments Company,Answer
4,49778,1,<strong>Operator</strong>\nLadies and gentleme...,2010-02-12,176660,3SBio Inc.,Presentation Section
...,...,...,...,...,...,...,...
12475891,1220890,45,So a key driver of the improvement last year a...,2017-05-11,220426,Zurich Insurance Group AG,Answer
12475892,1220890,43,"On the rate increase, I mean if you look at th...",2017-05-11,220426,Zurich Insurance Group AG,Answer
12475893,1220890,40,"So I'm going to apologize in advance, Ralph. I...",2017-05-11,220426,Zurich Insurance Group AG,Answer
12475894,1220890,37,"Yes, thanks. So there -- I mean, given what I ...",2017-05-11,220426,Zurich Insurance Group AG,Answer


In [ ]:
earnings_calls.dtypes

transcriptid                            int64
componentorder                          int64
componenttext                          object
mostimportantdateutc           datetime64[us]
gvkey                                  object
companyname                            object
transcriptcomponenttypename            object
dtype: object

In [ ]:
earnings_calls.sort_values(by=['mostimportantdateutc','transcriptid', 'componentorder'], inplace = True)

In [ ]:
#earnings_calls.to_parquet('datasets/Predictors/earnings_calls_sorted.parquet')

In [ ]:
earnings_calls = pd.read_parquet('datasets/Predictors/earnings_calls_sorted.parquet')

# Sentiment Analysis

In [ ]:
# Create a test dataframe to check our sentiment analysis
df = pd.DataFrame({
    'time': ['2021-07-15', '2020-07-12','2000-08-15'],
    'gvkey': [312, 12, 90],
    'text': ["Stock markets rallied after the company reported better-than-expected quarterly earnings. @salut #","Apple stock is expected to be stable",
             "Amazon stock will highly fall\n"]
})
df['time'] = pd.to_datetime(df['time'])
df

,time,gvkey,text
0,2021-07-15,312,Stock markets rallied after the company report...
1,2020-07-12,12,Apple stock is expected to be stable
2,2000-08-15,90,Amazon stock will highly fall\n


In [ ]:
# Convert to parquet as my class takes a parquet as input
df.to_parquet('datasets/Predictors/df.parquet')

In [ ]:
# Create an instance of my class with the created parquet file
reports = ReportsProcessing('datasets/Predictors/df.parquet', 'clean')
reports.data

,time,gvkey,text
0,2021-07-15,312,Stock markets rallied after the company report...
1,2020-07-12,12,Apple stock is expected to be stable
2,2000-08-15,90,Amazon stock will highly fall\n


In [ ]:
reports.clean_reports()

Cleaning Reports: 100%|██████████| 3/3 [00:00<00:00, 3175.10reports/s]


In [ ]:
reports.data

,time,gvkey,cleaned_text
0,2021-07-15,312,stock markets rallied after the company report...
1,2020-07-12,12,apple stock is expected to be stable
2,2000-08-15,90,amazon stock will highly fall


In [ ]:
# Perform sentiment analysis on my dataframe
reports.reports_sentiment_analysis()

Device set to use cpu
Detecting sentiments: 100%|██████████| 1/1 [00:00<00:00,  3.16it/s]

Sentiment Analysis Report:
	- Positive: 1 tweets (Avg. score: 0.93)
	- Neutral: 1 tweets (Avg. score: 0.94)
	- Negative: 1 tweets (Avg. score: 0.75)


In [ ]:
# Quick and More obvious example of how sentiment analysis works
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="ProsusAI/finbert", device=0)
text = "Apple stock is expected to be stable"
output = classifier(text)
print(output)

Device set to use cpu


[{'label': 'neutral', 'score': 0.9414412975311279}]


In [ ]:
reports.save_clean_data()

In [ ]:
sentiment = pd.read_parquet('./clean/sentiment_earning_calls.parquet')

In [ ]:
sentiment

,time,gvkey,cleaned_text,sentiment,sentiment_score
0,2021-07-15,312,stock markets rallied after the company report...,positive,0.934062
1,2020-07-12,12,apple stock is expected to be stable,neutral,0.941441
2,2000-08-15,90,amazon stock will highly fall,negative,0.752787


# Sentiment Analysis Earning Calls

In [ ]:
reports = ReportsProcessing('datasets/Predictors/earnings_calls_sorted.parquet', 'clean')
reports.data

,transcriptid,componentorder,componenttext,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename
198438,46682,1,"Good afternoon, everyone, and welcome to Merix...",2010-01-04,030247,Viasystems Corporation,Presentation Operator Message
1610990,46682,2,"Thank you. Happy New Year, everyone. In Octobe...",2010-01-04,030247,Viasystems Corporation,Presenter Speech
1610986,46682,3,"Thanks, Mike, and good afternoon, everyone. As...",2010-01-04,030247,Viasystems Corporation,Presenter Speech
1610982,46682,4,"Thanks, Kelly. As you can tell by our comments...",2010-01-04,030247,Viasystems Corporation,Presenter Speech
4020381,46682,5,[Operator Instructions] And our first question...,2010-01-04,030247,Viasystems Corporation,Question and Answer Operator Message
...,...,...,...,...,...,...,...
3620930,2164922,40,[Foreign Language],2020-12-30,161925,China Finance Online Co. Limited,Answer
3620861,2164922,41,Although the Chinese economy is leading the gr...,2020-12-30,161925,China Finance Online Co. Limited,Answer
1281137,2164922,42,[Operator Instructions] As there are no furthe...,2020-12-30,161925,China Finance Online Co. Limited,Question and Answer Operator Message
3620852,2164922,43,"Thank you, everybody, for attending China Fina...",2020-12-30,161925,China Finance Online Co. Limited,Answer


In [ ]:
reports.clean_reports()

Cleaning Reports: 100%|██████████| 12475896/12475896 [12:20<00:00, 16845.02reports/s]


In [ ]:
reports.data

,transcriptid,componentorder,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename,cleaned_text
198438,46682,1,2010-01-04,030247,Viasystems Corporation,Presentation Operator Message,"good afternoon, everyone, and welcome to merix..."
1610990,46682,2,2010-01-04,030247,Viasystems Corporation,Presenter Speech,"thank you. happy new year, everyone. in octobe..."
1610986,46682,3,2010-01-04,030247,Viasystems Corporation,Presenter Speech,"thanks, mike, and good afternoon, everyone. as..."
1610982,46682,4,2010-01-04,030247,Viasystems Corporation,Presenter Speech,"thanks, kelly. as you can tell by our comments..."
4020381,46682,5,2010-01-04,030247,Viasystems Corporation,Question and Answer Operator Message,and our first question comes from matt sheerin...
...,...,...,...,...,...,...,...
3620930,2164922,40,2020-12-30,161925,China Finance Online Co. Limited,Answer,
3620861,2164922,41,2020-12-30,161925,China Finance Online Co. Limited,Answer,although the chinese economy is leading the gr...
1281137,2164922,42,2020-12-30,161925,China Finance Online Co. Limited,Question and Answer Operator Message,as there are no further questions at this time...
3620852,2164922,43,2020-12-30,161925,China Finance Online Co. Limited,Answer,"thank you, everybody, for attending china fina..."


In [ ]:
reports.reports_sentiment_analysis()

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Detecting sentiments: 100%|██████████| 97468/97468 [6:52:15<00:00,  3.94it/s]


Sentiment Analysis Report:
	- Positive: 1728592 tweets (Avg. score: 0.76)
	- Neutral: 9875256 tweets (Avg. score: 0.84)
	- Negative: 872048 tweets (Avg. score: 0.76)


In [ ]:
reports.save_clean_data()

In [ ]:
df = pd.read_parquet('./clean/sentiment_earning_calls.parquet')

In [ ]:
df

,transcriptid,componentorder,mostimportantdateutc,gvkey,companyname,transcriptcomponenttypename,cleaned_text,sentiment,sentiment_score
0,46682,1,2010-01-04,030247,Viasystems Corporation,Presentation Operator Message,"good afternoon, everyone, and welcome to merix...",neutral,0.798825
1,46682,2,2010-01-04,030247,Viasystems Corporation,Presenter Speech,"thank you. happy new year, everyone. in octobe...",negative,0.792872
2,46682,3,2010-01-04,030247,Viasystems Corporation,Presenter Speech,"thanks, mike, and good afternoon, everyone. as...",positive,0.952724
3,46682,4,2010-01-04,030247,Viasystems Corporation,Presenter Speech,"thanks, kelly. as you can tell by our comments...",positive,0.891024
4,46682,5,2010-01-04,030247,Viasystems Corporation,Question and Answer Operator Message,and our first question comes from matt sheerin...,neutral,0.922422
...,...,...,...,...,...,...,...,...,...
12475891,2164922,40,2020-12-30,161925,China Finance Online Co. Limited,Answer,,neutral,0.424185
12475892,2164922,41,2020-12-30,161925,China Finance Online Co. Limited,Answer,although the chinese economy is leading the gr...,positive,0.906605
12475893,2164922,42,2020-12-30,161925,China Finance Online Co. Limited,Question and Answer Operator Message,as there are no further questions at this time...,neutral,0.937188
12475894,2164922,43,2020-12-30,161925,China Finance Online Co. Limited,Answer,"thank you, everybody, for attending china fina...",neutral,0.730117
